In [1]:
from utility import util, rating

In [2]:
conn = util.connectAlpaca()
universe = conn.list_assets(status='active', asset_class='us_equity')
print(len(universe))
universe[0]

Conneting...
9128


Asset({   'class': 'us_equity',
    'easy_to_borrow': True,
    'exchange': 'NYSE',
    'id': 'ec758e9f-503a-469e-a8b0-a3f27fe99316',
    'marginable': True,
    'name': 'Dynatrace, Inc.',
    'shortable': True,
    'status': 'active',
    'symbol': 'DT',
    'tradable': True})

In [3]:
universe = [{'exchange':x.exchange, 'symbol':x.symbol} for x in universe if x.shortable == True and x.tradable == True]
print(len(universe))
universe[0]

4466


{'exchange': 'NYSE', 'symbol': 'DT'}

In [4]:
%%time
from tradingview_ta import TA_Handler, Interval
import threading


def getSummary(i): 
    handler = TA_Handler()
    handler.set_screener_as_stock("america")
    handler.set_interval_as(Interval.INTERVAL_1_DAY)
    handler.set_exchange_as_crypto_or_stock(universe[i]['exchange'])
    handler.set_symbol_as(universe[i]['symbol'])
    try:
        universe[i]['rating'] = handler.get_analysis().summary
    except: 
        pass

threads = []
for i in range(len(universe)):
    x = threading.Thread(target=getSummary, args=(i,))
    threads.append(x)
    x.start()
# for thread in enumerate(threads):
#     thread.join()

print(len(universe))
universe[0]

4466
Wall time: 27.4 s


{'exchange': 'NYSE',
 'symbol': 'DT',
 'rating': {'RECOMMENDATION': 'BUY', 'BUY': 13, 'SELL': 5, 'NEUTRAL': 10}}

In [5]:
# split to five lists 
buyList = [
    x for x in universe 
    if 'rating' in x 
    and x['rating']['RECOMMENDATION'] == rating.BUY
]
buyList.sort(key=lambda x: x['rating'][rating.BUY], reverse = True)
buyStrongList = [
    x for x in universe 
    if 'rating' in x 
    and x['rating']['RECOMMENDATION'] == rating.STRONG_BUY
]
buyStrongList.sort(key=lambda x: x['rating'][rating.BUY], reverse = True)
nutrualList = [
    x for x in universe 
    if 'rating' in x 
    and x['rating']['RECOMMENDATION'] == rating.NEUTRAL
]
nutrualList.sort(key=lambda x: x['rating'][rating.NEUTRAL], reverse = True)
sellList = [
    x for x in universe 
    if 'rating' in x 
    and x['rating']['RECOMMENDATION'] == rating.SELL
]
sellList.sort(key=lambda x: x['rating'][rating.SELL], reverse = True)
sellStrongList = [
    x for x in universe 
    if 'rating' in x 
    and x['rating']['RECOMMENDATION'] == rating.STRONG_SELL
]
sellStrongList.sort(key=lambda x: x['rating'][rating.SELL], reverse = True)
print(len(buyList))
print(len(buyStrongList))
print(len(nutrualList))
print(len(sellList))
print(len(sellStrongList))
print(buyList[0])
print(buyStrongList[0])
print(nutrualList[0])
print(sellList[0])
print(sellStrongList[0])

2806
241
54
822
91
{'exchange': 'NASDAQ', 'symbol': 'AGNCP', 'rating': {'RECOMMENDATION': 'BUY', 'BUY': 19, 'SELL': 4, 'NEUTRAL': 5}}
{'exchange': 'NYSE', 'symbol': 'AMH', 'rating': {'RECOMMENDATION': 'STRONG_BUY', 'BUY': 20, 'SELL': 0, 'NEUTRAL': 8}}
{'exchange': 'NASDAQ', 'symbol': 'ACAD', 'rating': {'RECOMMENDATION': 'NEUTRAL', 'BUY': 8, 'SELL': 10, 'NEUTRAL': 10}}
{'exchange': 'AMEX', 'symbol': 'CQP', 'rating': {'RECOMMENDATION': 'SELL', 'BUY': 4, 'SELL': 19, 'NEUTRAL': 5}}
{'exchange': 'NASDAQ', 'symbol': 'CXDC', 'rating': {'RECOMMENDATION': 'STRONG_SELL', 'BUY': 0, 'SELL': 20, 'NEUTRAL': 8}}


In [6]:
# using each list's length as weight to create a final tradeable list
total = len(buyStrongList)+len(nutrualList)+len(sellStrongList)
finalList = []
finalList += buyStrongList[:int(len(buyStrongList)*(len(buyStrongList)/total))]
finalList += nutrualList[:int(len(nutrualList)*(len(nutrualList)/total))]
finalList += sellStrongList[:int(len(sellStrongList)*(len(sellStrongList)/total))]
len([x['symbol'] for x in finalList])

178